In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from AlgoTestFactory import *

# altf1 = AlgoTestFactory([0, 10000], 15, savePath="Combined_4_test2", storeAccuracies=True)

# altf1.run()

In [3]:
from SktWsegRWR_utf8 import *
import pickle
import ProbData
from ProbModels import *
import multiprocessing
import math
import json

Dataloader Started[Prob]...
Dataloader Finished[Prob]...


In [41]:
pb = ProbModels(fullCo_oc_mat = ProbData.fullCo_oc_mat, unigram_counts = ProbData.unigram_counts,
               cng2cngFullMat = ProbData.cng2cngFullMat, cng2index_dict = ProbData.cng2index_dict,
               w2w_samecng_fullmat=ProbData.w2w_samecng_fullmat, samecng_unigram_counts=ProbData.samecng_unigram_counts,
               v2c_fullMat = ProbData.v2c_fullMat)

goodFileDict = pickle.load(open('mergedGood_v2.p', 'rb'))
algo = SktWsegRWR(
            w2w_modelFunc = pb.get_w2w_mat, 
            t2t_modelFunc = pb.get_cng2cng_mat,
            v2c_modelFunc = pb.get_v2c_ranking,
            sameCng_modelFunc = pb.get_w2w_samecng_mat,
            partition=[0.25, 0.25, 0.25, 0.1]
        )

In [190]:
def loadSentence(fName, folderTag):
        # print('File: ', fName)
        try:
            dcsObj = pickleFixLoad('../Text Segmentation/DCS_pick/' + fName)           
            if folderTag == "C1020" :
                sentenceObj = pickleFixLoad('../TextSegmentation/corrected_10to20/' + fName)
            else:
                sentenceObj = pickleFixLoad('../TextSegmentation/Pickle_Files/' + fName)

        except (KeyError, EOFError, pickle.UnpicklingError) as e:
            return None, None
        return(sentenceObj, dcsObj)
    
def Accuracy(prediction, dcsObj):
#     solution = [rom_slp(c) for c in dcsObj.dcs_chunks]
    solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#     ac = 100*sum(list(map(lambda x: x in solution, prediction)))/len(prediction)
    ac = 100*sum(list(map(lambda x: x in prediction, solution)))/len(solution)
    return ac

In [ ]:
# lowfs = []
np.set_printoptions(precision=6)
# for f in list(goodFileDict.keys())[90:91]:
for f in list(goodFileDict.keys())[10:100]:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj)
        if(result != None):
            ac = Accuracy(result, dcsObj)
            if(ac < 40):
                print(result)
                lowfs.append(f)
                print(f, "\t", ac)

In [213]:
for f in lowfs[1:2]:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj)
        if(result != None):
            ac = Accuracy(result, dcsObj)
            print(result)
            print(f, "\t", ac)

['pUj', 'tad', 'sarva', 'rAjan', 'loka', 'a', 'mahat', 'pUj']
212470.p 	 33.333333333333336


In [216]:
SeeSentence(sentenceObj)

utpadyate mahArAjaH  sarvalokezu pUjitaH    
Analyzing  utpadyate
0 :  ut ['ut'] []
2 :  padyate ['pad_1'] [{'verb': ['pr. ps. sg. 3', 'pr. [4] md. sg. 3']}]
2 :  padya ['pad_1'] [{'indeclinable': ['abs.']}]
7 :  te ['tad', 'yuzmad'] [{'noun': ['acc. du. n.', 'nom. du. n.', 'nom. pl. m.', 'acc. du. f.', 'nom. du. f.']}, {'noun': ['g. sg. *', 'dat. sg. *']}]
Analyzing  mahArAjaḥ
0 :  mahar ['mahas'] [{'undetermined': ['und.']}]
0 :  mahA ['mahat'] [{'compound': ['iic.']}]
0 :  maha ['mah'] [{'verb': ['imp. [1] ac. sg. 2']}]
3 :  arAjas ['rAj_1'] [{'verb': ['impft. [1] ac. sg. 2']}]
3 :  ara ['ara'] [{'compound': ['iic.']}]
3 :  Ara ['Ara'] [{'compound': ['iic.']}]
3 :  arA ['ara'] [{'noun': ['nom. sg. f.']}]
3 :  ArA ['Ara'] [{'noun': ['nom. sg. f.']}]
3 :  ara ['f'] [{'verb': ['pft. ac. pl. 2', 'pft. ac. sg. 3', 'pft. ac. sg. 1']}]
3 :  a ['a'] [{'compound': ['iic.']}]
3 :  A ['A'] []
4 :  rAjas ['rAj_2'] [{'noun': ['acc. pl. m.', 'nom. pl. m.', 'g. sg. m.', 'abl. sg. m.', 'g. sg. n.',

In [217]:
SeeDCS(dcsObj)

utpadyate mahArAjaH  sarvalokezu pUjitaH    
[['utpad'], ['mahā', 'rāja'], ['sarva', 'loka'], ['pūjay']]
Lemmas: ['utpad', 'mahA', 'rAja', 'sarva', 'loka', 'pUjay']
[['-13'], ['2', '29'], ['3', '179'], ['-190']]


In [203]:
list(goodFileDict.keys())[13]

'327359.p'

In [209]:
lowfs

['179084.p',
 '212470.p',
 '141314.p',
 '312382.p',
 '371565.p',
 '20967.p',
 '178347.p',
 '289080.p',
 '334071.p',
 '407484.p',
 '209444.p']